<a href="https://colab.research.google.com/github/dominikmeyer95/academic-output/blob/feature%2Fapplied_numerical_finance/applied_numerical_finance/estimating_greeks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Applied Numerical Finance

* Group: Dominik Meyer, Mikhail Borovkov, Brage Bakken
* Assignment: 10 Finite difference approach for the Greeks

# Key Ressources & Notes

* Explain theoretical dimension of estimation methods in 1.1
* Estimate each Greek with all methods available and show potential difference in 1.2 (1.3)
* http://www.columbia.edu/~mh2078/MonteCarlo/MCS_Greeks.pdf
* https://www.codearmo.com/python-tutorial/options-trading-greeks-black-scholes
* Added "Pathwise Method" for now, in case we want to excel beyond demands
* Added "Second Order Greeks" for now, because task does not state exact scope

# Table of Contents

* 1 [Option Price Sensitivities](#chapter_1)
    * 1.1 [Estimation Methods](#section_1_1)
        * 1.1.1 [Closed Form Solutions](#subsection_1_1_1)
        * 1.1.2 [Finite Difference Approximation](#subsection_1_1_2)
        * 1.1.3 [The Pathwise Method](#subsection_1_1_3)
    * 1.2 [First Order Greeks](#section_1_2)
        * 1.2.1 [Delta: Option's Underlying Price Sensitivity](#subsection_1_2_1)
        * 1.2.2 [Vega: Option's Volatility Sensitivity](#subsection_1_2_2)
        * 1.2.3 [Theta: Option's Time Sensitivity](#subsection_1_2_3)
        * 1.2.4 [Rho: Option's Interest Rate Sensitivity](#subsection_1_2_4)
    * 1.3 [Second Order Greeks](#section_1_3)
        * 1.3.1 [Charm: Delta's Time Sensitivity](#subsection_1_3_1)
        * 1.3.2 [Gamma: Delta's Underlying Price Sensitivity](#subsection_1_3_2)
        * 1.3.3 [Vanna: Vega's Underlying Price Sensitivity](#subsection_1_3_3)
        * 1.3.4 [Vera: Vega's Interest Rate Sensitivity](#subsection_1_3_4)
        * 1.3.5 [Veta: Vega's Time Sensitivity](#subsection_1_3_5)
        * 1.3.6 [Volga: Vega's Volatility](#subsection_1_3_6)

In [2]:
import numpy as np
import pandas as pd
import scipy as sp
import plotly.express as px
import plotly.graph_objects as go

<a class="anchor" name="chapter_1"></a>
## 1 Option Price Sensitivities

<a class="anchor" name="section_1_1"></a>
### 1.1 Estimation Methods

<a class="anchor" name="subsection_1_1_1"></a>
### 1.1.1 Closed Form Solutions

<a class="anchor" name="subsection_1_1_2"></a>
### 1.1.2 Finite Difference Approximation

<a class="anchor" name="subsection_1_1_3"></a>
### 1.1.3 The Pathwise Method

<a class="anchor" name="section_1_2"></a>
### 1.2 First Order Greeks

<a class="anchor" name="subsection_1_2_1"></a>
### 1.2.1 Delta: Option's Underlying Price Sensitivity

<a class="anchor" name="subsection_1_2_2"></a>
### 1.2.2 Vega: Option's Volatility Sensitivity

<a class="anchor" name="subsection_1_2_3"></a>
### 1.2.3 Theta: Option's Time Sensitivity

<a class="anchor" name="subsection_1_2_4"></a>
### 1.2.4 Rho: Option's Interest Rate Sensitivity

<a class="anchor" name="section_1_3"></a>
### 1.3 Second Order Greeks

<a class="anchor" name="subsection_1_3_1"></a>
### 1.3.1 Charm: Delta's Time Sensitivity

<a class="anchor" name="subsection_1_3_2"></a>
### 1.3.2 Gamma: Delta's Underlying Price Sensitivity

<a class="anchor" name="subsection_1_3_3"></a>
### 1.3.3 Vanna: Vega's Underlying Price Sensitivity

<a class="anchor" name="subsection_1_3_4"></a>
### 1.3.4 Vera: Vega's Interest Rate Sensitivity

<a class="anchor" name="subsection_1_3_5"></a>
### 1.3.5 Veta: Vega's Time Sensitivity

<a class="anchor" name="subsection_1_3_6"></a>
### 1.3.6 Volga: Vega's Volatility Sensitivity